In [ ]:
import numpy as np
from scipy import fft
import xarray as xr
import pandas as pd
import os
from scipy import signal
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
latList = [-9, -8, -5, -2, 0, 2, 5, 8, 9]
lonList = [-95, -110, -125, -140, -155, -170, -180, 165]

ylen = len(latList)
xlen = len(lonList)

taskList = []

for latId  in range(ylen):
    for lonId in range(xlen):
        taskList.append([latList[latId], lonList[lonId]])

ntasks = len(taskList)


allDS = xr.Dataset()
dataCount = 0
fileCount = 0

for task in taskList:
    lat = task[0]
    lon = task[1]
    
    LAT = lat
    LON = lon
    
    if lat < 0:
        latUnits = 'S'
    else:
        latUnits = 'N'
    
    if lon < 0:
        lonUnits = 'W'
    else:
        lonUnits = 'E'
    
    LON = (LON+360)%360
    lat=abs(lat)
    lon=abs(lon)
    
    bFileName = f'../../../downloads/Buoy/extractedGZ/WINDS/T_{lat:02d}{latUnits}_{lon:03d}{lonUnits}_xrr_COARE3p5_2000_withRAIN.nc'
    wFileName = f'../../../downloads/Buoy/extractedGZ/WINDS/T_{lat:02d}{latUnits}_{lon:03d}{lonUnits}_xrr_COARE3p5_2000_withRAIN_2hrMeanVar.nc'
    if not os.path.isfile(bFileName):
        print(bFileName, 'not present')
        continue
    print(f'{lat:02d}{latUnits}_{lon:03d}{lonUnits}')
    ds = xr.open_dataset(bFileName)
    mask1 = ds.sel(HEIGHT=4)['WSPD_QC'].isin([1,2]).to_numpy()
    mask2 = ds.sel(HEIGHT=4)['WDIR_QC'].isin([1,2]).to_numpy()
    mask3 = ds.sel(DEPTH=1)['SST_QC'].isin([1,2]).to_numpy()
    mask4 = ds.sel(HEIGHT=3)['RELH_QC'].isin([1,2]).to_numpy()
    mask5 = ds.sel(HEIGHT=3)['AIRT_QC'].isin([1,2]).to_numpy()
    mask6 = ds.sel(HEIGHT=3)['RAIN_QC'].isin([1,2]).to_numpy()
    
    selectMask = np.logical_and(mask1, mask2)
    selectMask = np.logical_and(selectMask, mask3)
    selectMask = np.logical_and(selectMask, mask4)
    selectMask = np.logical_and(selectMask, mask5)
    selectMask = np.logical_and(selectMask, mask6)
    
    np_WSPD = ds.sel(HEIGHT=4)['WSPD'].to_numpy()
    np_WDIR = ds.sel(HEIGHT=4)['WDIR'].to_numpy()
    np_SST = ds.sel(DEPTH=1)['SST'].to_numpy()
    np_RELH = ds.sel(HEIGHT=3)['RELH'].to_numpy()
    np_AIRT = ds.sel(HEIGHT=3)['AIRT'].to_numpy()
    np_RAIN = ds.sel(HEIGHT=3)['RAIN'].to_numpy()
    
    
    mask1 = ~np.isnan(np_WSPD)
    mask2 = ~np.isnan(np_WDIR)
    mask3 = ~np.isnan(np_SST)
    mask4 = ~np.isnan(np_AIRT)
    mask5 = ~np.isnan(np_RELH)
    mask6 = ~np.isnan(np_RAIN)
    
    selectMask = np.logical_and(selectMask, mask1)
    selectMask = np.logical_and(selectMask, mask2)
    selectMask = np.logical_and(selectMask, mask3)
    selectMask = np.logical_and(selectMask, mask4)
    selectMask = np.logical_and(selectMask, mask5)
    selectMask = np.logical_and(selectMask, mask6)
    
    mask1 = ~(abs(np_WSPD) > 1000)
    mask2 = ~(abs(np_WDIR) > 1000)
    mask3 = ~(abs(np_SST) > 1000)
    mask4 = ~(abs(np_AIRT) > 1000)
    mask5 = ~(abs(np_RELH) > 1000)
    mask6 = ~(abs(np_RAIN) > 1000)
    
    selectMask = np.logical_and(selectMask, mask1)
    selectMask = np.logical_and(selectMask, mask2)
    selectMask = np.logical_and(selectMask, mask3)
    selectMask = np.logical_and(selectMask, mask4)
    selectMask = np.logical_and(selectMask, mask5)
    selectMask = np.logical_and(selectMask, mask6)
    
    indices = selectMask.nonzero()[0]
    
    ds = ds.isel(TIME=indices)
    
    time = pd.to_datetime(ds['TIME'].to_numpy())
    deltaTime = np.roll(time, -1) - time
    deltaTime = deltaTime
    deltaTime = np.array(deltaTime, dtype='timedelta64[s]')
    mask = np.logical_or(abs(deltaTime) > np.array([602], dtype='timedelta64[s]') , abs(deltaTime) < np.array([508], dtype='timedelta64[s]'))
    stopIndices = mask.nonzero()[0]+1
    stopIndices = np.concatenate(([0], stopIndices))
    
    
    ds['WDIR'] = ((-ds['WDIR'] + 90)+360)%360
    ds['cosWDIR'] = np.cos(np.deg2rad(ds['WDIR']))
    ds['sinWDIR'] = np.sin(np.deg2rad(ds['WDIR']))
    ds['U10N_x'] = ds.WSPD_10N.sel(HEIGHT=[10]) * ds.cosWDIR.sel(HEIGHT=4)
    ds['U10N_y'] = ds.WSPD_10N.sel(HEIGHT=[10]) * ds.sinWDIR.sel(HEIGHT=4)
    
    
    vars = ['WSPD', 'WSPD_10N', 'SST', 'AIRT', 'RELH', 'U10N_x', 'U10N_y', 'cosWDIR', 'sinWDIR', 'RAIN']
    
    for var in vars:
        print(var)
        if var in ['WSPD', 'cosWDIR', 'sinWDIR']:
            xarr = ds[var].sel(HEIGHT=4)
            xarr = xarr.drop_vars('HEIGHT')
        elif var in ['WSPD_10N', 'U10N_x', 'U10N_y']:
            xarr = ds[var].sel(HEIGHT=10)
            xarr = xarr.drop_vars('HEIGHT')
        elif var == 'SST':
            xarr = ds[var].sel(DEPTH=1)
            xarr = xarr.drop_vars('DEPTH')
        else:
            xarr = ds[var].sel(HEIGHT=3)
            xarr = xarr.drop_vars('HEIGHT')
        ksize = 12
        np_arr = xarr.to_numpy()
        kernel = np.ones((ksize), dtype=float)
        kernel /= sum(kernel)
        arr_bar = signal.convolve(np_arr, kernel, mode='same')
        arrSq_bar = signal.convolve((np_arr)**2, kernel, mode='same') #
        arrVar = arrSq_bar - (arr_bar)**2
        arrStd = np.sqrt(arrVar)
        hksize = int(ksize//2)
        arrLen = len(np_arr)
        for index in stopIndices:
            start = index - hksize-1
            end = index + hksize + 2
            if start < 0:
                start = 0
            if end>arrLen:
                end =arrLen
            arr_bar[start:end] = np.nan
            arrStd[start:end] = np.nan
            
        std = xr.DataArray(arrStd, dims = ['TIME'],
                           coords = {'TIME': xarr['TIME']},
                           attrs = {'statistic': f'running std. dev with {ksize}*10 minutes using convolution'})
    
        mean = xr.DataArray(arr_bar, dims = ['TIME'],
                           coords = {'TIME': xarr['TIME']},
                           attrs = {'statistic': f'running mean with {ksize}*10 minutes using convolution'})
        
        ds[f'mean_{var}'] = mean
        ds[f'std_{var}'] = std
    
    ds.to_netcdf(wFileName, unlimited_dims='TIME')
    ds.close()


In [ ]:
lat = 2
lon = -140

LAT = lat
LON = lon

if lat < 0:
    latUnits = 'S'
else:
    latUnits = 'N'

if lon < 0:
    lonUnits = 'W'
else:
    lonUnits = 'E'

LON = (LON+360)%360
lat=abs(lat)
lon=abs(lon)

bFileName = f'../../../downloads/Buoy/extractedGZ/WINDS/T_{lat:02d}{latUnits}_{lon:03d}{lonUnits}_xrr_COARE3p5_2000_withRAIN.nc'
# if not os.path.isfile(bFileName):
#     print(bFileName, 'not present')
#     continue
ds = xr.open_dataset(bFileName)
mask1 = ds.sel(HEIGHT=4)['WSPD_QC'].isin([1,2]).to_numpy()
mask2 = ds.sel(HEIGHT=4)['WDIR_QC'].isin([1,2]).to_numpy()
mask3 = ds.sel(DEPTH=1)['SST_QC'].isin([1,2]).to_numpy()
mask4 = ds.sel(HEIGHT=3)['RELH_QC'].isin([1,2]).to_numpy()
mask5 = ds.sel(HEIGHT=3)['AIRT_QC'].isin([1,2]).to_numpy()
mask6 = ds.sel(HEIGHT=3)['RAIN_QC'].isin([1,2]).to_numpy()

selectMask = np.logical_and(mask1, mask2)
selectMask = np.logical_and(selectMask, mask3)
selectMask = np.logical_and(selectMask, mask4)
selectMask = np.logical_and(selectMask, mask5)
selectMask = np.logical_and(selectMask, mask6)

np_WSPD = ds.sel(HEIGHT=4)['WSPD'].to_numpy()
np_WDIR = ds.sel(HEIGHT=4)['WDIR'].to_numpy()
np_SST = ds.sel(DEPTH=1)['SST'].to_numpy()
np_RELH = ds.sel(HEIGHT=3)['RELH'].to_numpy()
np_AIRT = ds.sel(HEIGHT=3)['AIRT'].to_numpy()
np_RAIN = ds.sel(HEIGHT=3)['RAIN'].to_numpy()


mask1 = ~np.isnan(np_WSPD)
mask2 = ~np.isnan(np_WDIR)
mask3 = ~np.isnan(np_SST)
mask4 = ~np.isnan(np_AIRT)
mask5 = ~np.isnan(np_RELH)
mask6 = ~np.isnan(np_RAIN)

selectMask = np.logical_and(selectMask, mask1)
selectMask = np.logical_and(selectMask, mask2)
selectMask = np.logical_and(selectMask, mask3)
selectMask = np.logical_and(selectMask, mask4)
selectMask = np.logical_and(selectMask, mask5)
selectMask = np.logical_and(selectMask, mask6)

mask1 = ~(abs(np_WSPD) > 1000)
mask2 = ~(abs(np_WDIR) > 1000)
mask3 = ~(abs(np_SST) > 1000)
mask4 = ~(abs(np_AIRT) > 1000)
mask5 = ~(abs(np_RELH) > 1000)
mask6 = ~(abs(np_RAIN) > 1000)

selectMask = np.logical_and(selectMask, mask1)
selectMask = np.logical_and(selectMask, mask2)
selectMask = np.logical_and(selectMask, mask3)
selectMask = np.logical_and(selectMask, mask4)
selectMask = np.logical_and(selectMask, mask5)
selectMask = np.logical_and(selectMask, mask6)

indices = selectMask.nonzero()[0]

ds = ds.isel(TIME=indices)

time = pd.to_datetime(ds['TIME'].to_numpy())
deltaTime = np.roll(time, -1) - time
deltaTime = deltaTime
deltaTime = np.array(deltaTime, dtype='timedelta64[s]')
mask = np.logical_or(abs(deltaTime) > np.array([602], dtype='timedelta64[s]') , abs(deltaTime) < np.array([508], dtype='timedelta64[s]'))
stopIndices = mask.nonzero()[0]+1
stopIndices = np.concatenate(([0], stopIndices))


ds['WDIR'] = ((-ds['WDIR'] + 90)+360)%360
ds['cosWDIR'] = np.cos(np.deg2rad(ds['WDIR']))
ds['sinWDIR'] = np.sin(np.deg2rad(ds['WDIR']))
ds['U10N_x'] = ds.WSPD_10N.sel(HEIGHT=[10]) * ds.cosWDIR.sel(HEIGHT=4)
ds['U10N_y'] = ds.WSPD_10N.sel(HEIGHT=[10]) * ds.sinWDIR.sel(HEIGHT=4)


vars = ['WSPD', 'WSPD_10N', 'SST', 'AIRT', 'RELH', 'U10N_x', 'U10N_y', 'cosWDIR', 'sinWDIR', 'RAIN']

for var in vars:
    print(var)
    if var in ['WSPD', 'cosWDIR', 'sinWDIR']:
        xarr = ds[var].sel(HEIGHT=4)
        xarr = xarr.drop_vars('HEIGHT')
    elif var in ['WSPD_10N', 'U10N_x', 'U10N_y']:
        xarr = ds[var].sel(HEIGHT=10)
        xarr = xarr.drop_vars('HEIGHT')
    elif var == 'SST':
        xarr = ds[var].sel(DEPTH=1)
        xarr = xarr.drop_vars('DEPTH')
    else:
        xarr = ds[var].sel(HEIGHT=3)
        xarr = xarr.drop_vars('HEIGHT')
    ksize = 12
    np_arr = xarr.to_numpy()
    kernel = np.ones((ksize), dtype=float)
    kernel /= sum(kernel)
    arr_bar = signal.convolve(np_arr, kernel, mode='same')
    arrSq_bar = signal.convolve((np_arr)**2, kernel, mode='same') #
    arrVar = arrSq_bar - (arr_bar)**2
    arrStd = np.sqrt(arrVar)
    hksize = int(ksize//2)
    arrLen = len(np_arr)
    for index in stopIndices:
        start = index - hksize-1
        end = index + hksize + 2
        if start < 0:
            start = 0
        if end>arrLen:
            end =arrLen
        arr_bar[start:end] = np.nan
        arrStd[start:end] = np.nan
        
    std = xr.DataArray(arrStd, dims = ['TIME'],
                       coords = {'TIME': xarr['TIME']},
                       attrs = {'statistic': f'running std. dev with {ksize}*10 minutes using convolution'})

    mean = xr.DataArray(arr_bar, dims = ['TIME'],
                       coords = {'TIME': xarr['TIME']},
                       attrs = {'statistic': f'running mean with {ksize}*10 minutes using convolution'})
    
    ds[f'mean_{var}'] = mean
    ds[f'std_{var}'] = std

ds.to_netcdf(wfileName, unlimited_dims='TIME')

In [ ]:
np.sum(mask1), np.sum(mask2), np.sum(mask3), np.sum(mask4), np.sum(mask5), np.sum(mask6)

In [ ]:
stopIndices

In [ ]:
ds = ds.chunk({'TIME':-1})

In [ ]:
ds

In [ ]:
def getMean(xarr, fillIndices, ksize = 6,):
    np_arr = np.squeeze(xarr)
    kernel = np.ones((ksize), dtype=float)
    kernel /= sum(kernel)
    arr_bar = signal.convolve(np_arr, kernel, mode='same')
    hksize = int(ksize//2)
    arrLen = len(np_arr)
    for index in fillIndices:
        start = index - hksize-1
        end = index + hksize + 2
        if start < 0:
            start = 0
        if end>arrLen:
            end =arrLen
        arr_bar[start:end] = np.nan
        
    mean = xr.DataArray(arr_bar, dims=['TIME'], 
                        coords = {'TIME': xarr['TIME']},
                        attrs = {'statistic': f'running mean with {ksize}*10 minutes using convolution'})
    return mean


def getStd(xarr, fillIndices, ksize = 6,):
    np_arr = np.squeeze(xarr)
    kernel = np.ones((ksize), dtype=float)
    kernel /= sum(kernel)
    arr_bar = signal.convolve(np_arr, kernel, mode='same')
    arrSq_bar = signal.convolve(np_arr*np_arr, kernel, mode='same')
    arrVar = arrSq_bar - arr_bar*arr_bar
    arrStd = np.sqrt(arrVar)
    hksize = int(ksize//2)
    arrLen = len(np_arr)
    for index in fillIndices:
        start = index - hksize-1
        end = index + hksize + 2
        if start < 0:
            start = 0
        if end>arrLen:
            end =arrLen
        arrStd[start:end] = np.nan
        
    std = xr.DataArray(arrStd, dims = ['TIME'],
                       coords = {'TIME': xarr['TIME']},
                       attrs = {'statistic': f'running std. dev with {ksize}*10 minutes using convolution'})
    
    return std


def getMeanAndStd(xarr, fillIndices, ksize = 6,):
    np_arr = np.squeeze(xarr)
    kernel = np.ones((ksize), dtype=float)
    kernel /= sum(kernel)
    arr_bar = signal.convolve(np_arr, kernel, mode='same')
    arrSq_bar = signal.convolve(np_arr*np_arr, kernel, mode='same')
    arrVar = arrSq_bar - arr_bar*arr_bar
    arrStd = np.sqrt(arrVar)
    hksize = int(ksize//2)
    arrLen = len(np_arr)
    for index in fillIndices:
        start = index - hksize-1
        end = index + hksize + 2
        if start < 0:
            start = 0
        if end>arrLen:
            end =arrLen
        arr_bar[start:end] = np.nan
        
    mean = xr.DataArray(arr_bar, dims=['TIME'], 
                        coords = {'TIME': xarr['TIME']},
                        attrs = {'statistic': f'running mean with {ksize}*10 minutes using convolution'})
    std = xr.DataArray(arrStd, dims = ['TIME'],
                       coords = {'TIME': xarr['TIME']},
                       attrs = {'statistic': f'running std. dev with {ksize}*10 minutes using convolution'})
    return mean, std
                       
    

In [ ]:
ds['WDIR'] = ((-ds['WDIR'] + 90)+360)%360
ds['cosWDIR'] = np.cos(np.deg2rad(ds['WDIR']))
ds['sinWDIR'] = np.sin(np.deg2rad(ds['WDIR']))
ds['U10N_x'] = ds.WSPD_10N.sel(HEIGHT=[10]) * ds.cosWDIR.sel(HEIGHT=4)
ds['U10N_y'] = ds.WSPD_10N.sel(HEIGHT=[10]) * ds.sinWDIR.sel(HEIGHT=4)

In [ ]:
ds.U10N_x

In [ ]:
ds.WSPD

In [ ]:
#ds.U10N_x.sel(HEIGHT=10).plot(x='TIME')

In [ ]:
vars = ['WSPD', 'WSPD_10N', 'SST', 'AIRT', 'RELH', 'U10N_x', 'U10N_y', 'cosWDIR', 'sinWDIR', 'RAIN']

for var in vars:
    print(var)
    if var in ['WSPD', 'cosWDIR', 'sinWDIR']:
        xarr = ds[var].sel(HEIGHT=4)
        xarr = xarr.drop_vars('HEIGHT')
    elif var in ['WSPD_10N', 'U10N_x', 'U10N_y']:
        xarr = ds[var].sel(HEIGHT=10)
        xarr = xarr.drop_vars('HEIGHT')
    elif var == 'SST':
        xarr = ds[var].sel(DEPTH=1)
        xarr = xarr.drop_vars('DEPTH')
    else:
        xarr = ds[var].sel(HEIGHT=3)
        xarr = xarr.drop_vars('HEIGHT')
    ksize = 12
    np_arr = xarr.to_numpy()
    kernel = np.ones((ksize), dtype=float)
    kernel /= sum(kernel)
    arr_bar = signal.convolve(np_arr, kernel, mode='same')
    arrSq_bar = signal.convolve((np_arr)**2, kernel, mode='same') #
    arrVar = arrSq_bar - (arr_bar)**2
    arrStd = np.sqrt(arrVar)
    hksize = int(ksize//2)
    arrLen = len(np_arr)
    for index in stopIndices:
        start = index - hksize-1
        end = index + hksize + 2
        if start < 0:
            start = 0
        if end>arrLen:
            end =arrLen
        arr_bar[start:end] = np.nan
        arrStd[start:end] = np.nan
        
    std = xr.DataArray(arrStd, dims = ['TIME'],
                       coords = {'TIME': xarr['TIME']},
                       attrs = {'statistic': f'running std. dev with {ksize}*10 minutes using convolution'})

    mean = xr.DataArray(arr_bar, dims = ['TIME'],
                       coords = {'TIME': xarr['TIME']},
                       attrs = {'statistic': f'running mean with {ksize}*10 minutes using convolution'})
    
    ds[f'mean_{var}'] = mean
    ds[f'std_{var}'] = std

In [ ]:
plt.plot(ds['RAIN'][17100:17200])
plt.plot(ds['std_RAIN'][17100:17200])
plt.plot(ds['mean_RAIN'][17100:17200])

In [ ]:
# xarr = ds['SST'].sel(DEPTH=1)
# xarr = xarr.drop_vars('DEPTH')
# np_arr = xarr.to_numpy()
ksize = 12
kernel = np.ones((ksize), dtype=float)
kernel /= sum(kernel)
K = 0
arr_bar = signal.convolve(np_arr-K, kernel, mode='same')
arrSq_bar = signal.convolve((np_arr-K)**2, kernel, mode='same') 
for index in stopIndices:
        start = index - hksize-1
        end = index + hksize + 2
        if start < 0:
            start = 0
        if end>arrLen:
            end =arrLen
        arr_bar[start:end] = np.nan
        arrSq_bar[start:end] = np.nan

In [ ]:
plt.plot(arrSq_bar - arr_bar**2)

In [ ]:
np.nanmin(arrSq_bar - arr_bar**2)

In [ ]:
# #ds.WSPD.sel(HEIGHT=4).isel(TIME=slice(1,500)).plot(x='TIME')
# ds.RAIN.isel(TIME=slice(1,100)).plot(x='TIME')
# ds.std_WSPD.isel(TIME=slice(1,100)).plot(x='TIME')

In [ ]:
timeArr = ds.TIME.to_numpy()

In [ ]:
ds

In [ ]:
#RAIN = xr.where(np.logical_or(abs(ds.RAIN.sel(HEIGHT=3)) > 1000, ds.RAIN.sel(HEIGHT=3) < 0), np.nan, ds.RAIN.sel(HEIGHT=3))

In [ ]:
RAIN = xr.where(abs(ds.RAIN.sel(HEIGHT=3)) > 1000, np.nan, ds.RAIN.sel(HEIGHT=3)).to_numpy()
RAIN_0 = (RAIN - np.nanmean(RAIN))/np.nanstd(RAIN)

from sklearn.preprocessing import RobustScaler
scaler = RobustScaler()
RAIN_1 = np.log(RAIN - np.nanmin(RAIN) + np.exp(1))
RAIN_1 = scaler.fit_transform(RAIN_1.reshape(-1,1))


In [ ]:
plt.plot(RAIN_0)


In [ ]:
test = np.log10(RAIN - np.nanmin(RAIN) + 10)
#test = (test - np.nanmean(test)/np.nanstd(test))
plt.plot(test)

In [ ]:
plt.hist(RAIN, bins=200)
plt.yscale('log')

In [ ]:
plt.plot()

In [ ]:
#RAIN.isel(TIME=slice(1,1000)).plot(x='TIME')
#ds.std_WSPD.isel(TIME=slice(1,500)).plot(x='TIME')


In [ ]:
# RAIN.plot(x='TIME')

In [ ]:
ds.std_WSPD.plot(x='TIME')

In [ ]:
std = ds.std_WSPD.to_numpy()
std = std-np.nanstd(std)/np.nanstd(std)

In [ ]:
plt.figure(figsize=(20,5))
plt.plot(ds['TIME'], RAIN*0.1)
#plt.plot(ds['TIME'], std, alpha=0.5)
plt.plot(ds['TIME'], ds.std_sinWDIR, alpha=0.75)
#plt.plot(ds['TIME'], ds.RELH.sel(HEIGHT=3), alpha=0.5)
#plt.plot(ds['TIME'], ds.mean_RELH, alpha=0.5)

In [ ]:
plt.plot(ds['TIME'], ds.std_RELH, alpha=0.5)

In [ ]:
ds['mean_WVEl'] = xr.DataArray(np.sqrt(ds['mean_U10N_x']**2 + ds['mean_U10N_y']**2), 
                               dims = ('TIME'), 
                               coords = {'TIME':ds['TIME']})

In [ ]:
test = ds.mean_WVEl.to_numpy()
test = (test - np.nanmean(test))/np.nanstd(test)

In [ ]:
plt.figure(figsize=(20,5))
plt.plot(ds['TIME'], test*0.25)#, alpha=0.75)

In [ ]:
ds